# SpecViz Demo Notebook
## Basic Machinery

In [ ]:
import astropy.units as u
import specutils

In [ ]:
# Suppress warnings
import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore")

## Create SpecViz via Helper

In [ ]:
from jdaviz import SpecViz
specviz = SpecViz()

## Load a file to SpecViz

In [ ]:
from astropy.utils.data import download_file
fn = download_file('https://data.sdss.org/sas/dr14/sdss/spectro/redux/26/spectra/0751/spec-0751-52251-0160.fits', cache=True)
specviz.load_spectrum(fn, "myfile", format="SDSS-III/IV spec")

## Display SpecViz

In [ ]:
specviz.app

## Subset Retrieval
User Task: Select a subset in the viewer above

After defining regions in your spectra, you can retrieve your subsets through different means. To retrieve your subset by name:

In [ ]:
specviz.get_spectra('Subset 1')

Or you can retrieve all defined regions/subsets:

In [ ]:
specviz.app.get_subsets_from_viewer('spectrum-viewer')

## Screenshot Saving

In [ ]:
specviz.app.get_viewer("spectrum-viewer").figure.save_png()

## Panning/Zooming in SpecViz
### Limit Methods
You can use the methods x_limits() and y_limits() to modify the field of view of SpecViz. You can provide it a scalar (which assumes the units of the loaded spectra), an Astropy Quantity, or 'auto' to automatically scale.

In [ ]:
specviz.x_limits()
specviz.x_limits(650*u.nm,750*u.nm)
specviz.y_limits('auto', 110.0)

### Autoscale Methods
You can also quickly return to the default zoom using autoscale_x() and autoscale_y()

In [ ]:
specviz.autoscale_x()
specviz.autoscale_y()

# Science Usecase

## Retrieving your region from SpecViz

In [ ]:
# THIS SHOULD BE UPDATED WHEN 159/161 are in!
spec=spectra
ss = specviz.app.data_collection.subset_groups[-1].subsets[0].subset_state
line_region = specutils.SpectralRegion(ss.lo*spec.spectral_axis.unit, ss.hi*spec.spectral_axis.unit)
line_region

## Create a model 

In [ ]:
from astropy.modeling import models
from specutils.fitting import fit_lines
from  specutils import manipulation

line_model = models.Gaussian1D(mean=(line_region.lower + line_region.upper)/2, 
                               stddev=3, 
                               amplitude=1000) + models.Const1D(200)

#fit that model to the selected region

# after a bug fix, this should be:
# fit_lines(spec, line_model, window=line_region)

extracted = manipulation.extract_region(spec, line_region)
extracted.mask[:] = False
fitted_line = fit_lines(extracted, line_model)
#fitted_line = fit_lines(spec, line_model)
fitted_line

## Plot the results

In [ ]:
from matplotlib import pyplot as plt
plt.plot(spec.spectral_axis.value, spec.flux.value)
plt.plot(spec.spectral_axis, fitted_line(spec.spectral_axis))

plt.xlim(6500, 7200)
plt.ylim(0, 110)

# integral of just the gaussian part of the line *might* be what you want as "line flux"?

from scipy.integrate import quad
quad(fitted_line.unitless_model.left, 6500, 6700)[0]*fitted_line.return_units